In [61]:
import pandas as pd
from torch.utils.data import DataLoader,Dataset
from transformers import  MT5ForConditionalGeneration,T5Tokenizer
from transformers import DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split

In [69]:
class TranslationDataset(Dataset):

    def __init__(self,data_frame,tokenizer_name='google/mt5-base') -> None:
        super().__init__()
        self.dataframe = data_frame
        self.tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)

    def __len__(self):
        return self.dataframe.shape[0]
    def __getitem__(self, index):
        tokenized_data = self.tokenizer(self.dataframe.iloc[index,0],text_target=self.dataframe.iloc[index,1], return_tensors="pt")
        
        tokenized_data['input_ids'] = tokenized_data['input_ids'].squeeze(0)
        tokenized_data['attention_mask'] = tokenized_data['attention_mask'].squeeze(0)
        tokenized_data['labels'] = tokenized_data['labels'].squeeze(0)
        return tokenized_data


In [ ]:

tokenizer_name = "google/mt5-base"
translation_model = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")
t5_tokenizer =  T5Tokenizer.from_pretrained(tokenizer_name)

data_collator = DataCollatorForSeq2Seq(t5_tokenizer, model=translation_model, return_tensors="pt")

In [ ]:
def collate_fn(batch_data):
    print(batch_data)
    

In [65]:

data_frame = pd.read_csv("./data/EN-DE.txt", sep='\t',header=0, names=['src', 'trg', 'c1','c2','c3','c4','c5','c6'])[:100]
train_df , valid_df = train_test_split(data_frame,test_size=0.07)

In [67]:
train_dataset = TranslationDataset(train_df)
valid_dataset = TranslationDataset(valid_df)

train_dataloader = DataLoader(train_dataset,6,collate_fn=collate_fn,shuffle=True)

In [68]:
f = next(iter(train_dataloader))

[{'input_ids': tensor([  1494,    259, 117741,    259,  22405,    390,    259,  64489,    281,
          2476,    259,  21202,   1421,    305,   3867,    484,  70129,    281,
           751,  39359,   9629,    260,      1]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'labels': tensor([ 10092,   1598,  64943,    856,    281,  23310,    259,    266,  16845,
         17434,    278,  89899,   2682,    472,    259,  35935,    335,  24753,
          1504,    281,    384,   2930, 112862,  23708,    260,      1])}, {'input_ids': tensor([39359,   304,   287, 22407,     1]), 'attention_mask': tensor([1, 1, 1, 1, 1]), 'labels': tensor([112862,    442,    259, 125908,      1])}, {'input_ids': tensor([  259, 29655,   358,   267,     1]), 'attention_mask': tensor([1, 1, 1, 1, 1]), 'labels': tensor([   281, 167041,   1283,  24588,   2081, 100104,    286,    259, 174828,
           267,      1])}, {'input_ids': tensor([5366,  333,    1]), 'attentio

In [57]:

tokenized_text = []
for i in range(len(new_df)):
    tokenized_text.append(tokenizer(new_df.iloc[i,0],text_target=new_df.iloc[i,1], return_tensors="pt"))


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
for i in range(len(tokenized_text)):
    ith_dict = tokenized_text[i]
    ith_dict['input_ids'] = ith_dict['input_ids'].squeeze(0)
    ith_dict['labels'] = ith_dict['labels'].squeeze(0)
    ith_dict['attention_mask'] = ith_dict['attention_mask'].squeeze(0)
    tokenized_text[i] = ith_dict

In [46]:
tokenized_text[0]

{'input_ids': tensor([ 304,  670,  259, 3957, 3749,    1]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1]), 'labels': tensor([ 5529,  5383, 14703,  3749,     1])}

In [47]:
from transformers import MT5ForConditionalGeneration



data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

In [48]:
batch = data_collator([tokenized_text[i] for i in range(len(tokenized_text))])

In [51]:
batch['labels'][0]

tensor([ 5529,  5383, 14703,  3749,     1,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 